# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,thompson,55.7435,-97.8558,14.09,82,100,4.63,CA,1691532069
1,1,asenovgrad,42.0167,24.8667,17.84,63,48,0.51,BG,1691532928
2,2,ha'il,27.5219,41.6907,32.01,15,0,2.06,SA,1691532929
3,3,leo,11.1000,-2.1000,23.10,92,100,2.36,BF,1691532929
4,4,amursk,50.2261,136.8994,16.96,63,100,3.73,RU,1691532929


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    tiles="OSM", 
    s="Humidity", 
    color="City", 
    alpha=0.5,
    frame_width=700,
    frame_height=500)



# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"] < 26) & (city_data_df["Max Temp"] > 23) & 
                                (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_cities_filtered = ideal_cities.dropna(how="any")

# Display sample data
ideal_cities_filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,hammerfest,70.6634,23.6821,23.25,73,0,2.06,NO,1691532934
180,180,piterka,50.6803,47.4454,24.12,41,0,3.23,RU,1691533047
220,220,felahiye,39.0906,35.5672,25.41,54,0,2.32,TR,1691533071
244,244,souda,35.4844,24.0744,24.63,78,0,0.00,GR,1691533082
442,442,paphos,34.7667,32.4167,24.88,78,0,3.09,CY,1691533035
510,510,darwin,-12.4611,130.8418,23.83,68,0,2.68,AU,1691533258
557,557,korla,41.7597,86.1469,24.88,40,0,1.51,CN,1691533290


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_filtered.copy()
hotel_df = hotel_df.drop(["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"], axis="columns")

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
11,hammerfest,70.6634,23.6821,73,NO,
180,piterka,50.6803,47.4454,41,RU,
220,felahiye,39.0906,35.5672,54,TR,
244,souda,35.4844,24.0744,78,GR,
442,paphos,34.7667,32.4167,78,CY,
510,darwin,-12.4611,130.8418,68,AU,
557,korla,41.7597,86.1469,40,CN,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hammerfest - nearest hotel: Thon Hotel Hammerfest
piterka - nearest hotel: No hotel found
felahiye - nearest hotel: No hotel found
souda - nearest hotel: Panos Village
paphos - nearest hotel: Mayfair Hotel
darwin - nearest hotel: Mantra Pantanas Darwin
korla - nearest hotel: Silver Star Hotel


,City,Lat,Lng,Humidity,Country,Hotel Name
11,hammerfest,70.6634,23.6821,73,NO,Thon Hotel Hammerfest
180,piterka,50.6803,47.4454,41,RU,No hotel found
220,felahiye,39.0906,35.5672,54,TR,No hotel found
244,souda,35.4844,24.0744,78,GR,Panos Village
442,paphos,34.7667,32.4167,78,CY,Mayfair Hotel
510,darwin,-12.4611,130.8418,68,AU,Mantra Pantanas Darwin
557,korla,41.7597,86.1469,40,CN,Silver Star Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    s="Humidity",
    color="City",
    alpha=0.5,
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)